In [4]:
require 'elasticsearch'
  
client = Elasticsearch::Client.new url: 'http://localhost:9200/', log: true

client.create index: 'json_index',
              type: 'json',
              id: '3',
              body: {
                request: {
                  method: "GET",
                  url: "/api/mytest3"
                },
                response: {
                  status: 200,
                  body: "More content"
                }
              }
              
            

{"_index"=>"json_index", "_type"=>"json", "_id"=>"3", "_version"=>1, "result"=>"created", "_shards"=>{"total"=>2, "successful"=>1, "failed"=>0}, "_seq_no"=>0, "_primary_term"=>1}

In [5]:
result = client.search q: '_index:json_index'

{"took"=>33, "timed_out"=>false, "_shards"=>{"total"=>6, "successful"=>6, "skipped"=>0, "failed"=>0}, "hits"=>{"total"=>3, "max_score"=>1.0, "hits"=>[{"_index"=>"json_index", "_type"=>"json", "_id"=>"2", "_score"=>1.0, "_source"=>{"request"=>{"method"=>"GET", "url"=>"/api/mytest2"}, "response"=>{"status"=>200, "body"=>"More content"}}}, {"_index"=>"json_index", "_type"=>"json", "_id"=>"1", "_score"=>1.0, "_source"=>{"request"=>{"method"=>"GET", "url"=>"/api/mytest"}, "response"=>{"status"=>200, "body"=>"More content"}}}, {"_index"=>"json_index", "_type"=>"json", "_id"=>"3", "_score"=>1.0, "_source"=>{"request"=>{"method"=>"GET", "url"=>"/api/mytest3"}, "response"=>{"status"=>200, "body"=>"More content"}}}]}}

In [6]:
filter_results = result['hits']['hits']
  .map do |doc|
    doc['_source']
  end
  .select do |doc|
    doc['request'] and doc['response']
  end

[{"request"=>{"method"=>"GET", "url"=>"/api/mytest2"}, "response"=>{"status"=>200, "body"=>"More content"}}, {"request"=>{"method"=>"GET", "url"=>"/api/mytest"}, "response"=>{"status"=>200, "body"=>"More content"}}, {"request"=>{"method"=>"GET", "url"=>"/api/mytest3"}, "response"=>{"status"=>200, "body"=>"More content"}}]

In [8]:
filter_results.each do |doc|
  puts doc
end

{"request"=>{"method"=>"GET", "url"=>"/api/mytest"}, "response"=>{"status"=>200, "body"=>"More content"}}
{"request"=>{"method"=>"GET", "url"=>"/api/mytest"}, "response"=>{"status"=>200, "body"=>"More content"}}
{"request"=>{"method"=>"GET", "url"=>"/api/mytest"}, "response"=>{"status"=>200, "body"=>"More content"}}


[{"request"=>{"method"=>"GET", "url"=>"/api/mytest"}, "response"=>{"status"=>200, "body"=>"More content"}}, {"request"=>{"method"=>"GET", "url"=>"/api/mytest"}, "response"=>{"status"=>200, "body"=>"More content"}}, {"request"=>{"method"=>"GET", "url"=>"/api/mytest"}, "response"=>{"status"=>200, "body"=>"More content"}}]

In [16]:
f = File.new("request-mock.json", "w")

#<File:request-mock.json>

In [17]:
mappings = { :mappings => filter_results } 

{:mappings=>[{"request"=>{"method"=>"GET", "url"=>"/api/mytest"}, "response"=>{"status"=>200, "body"=>"More content"}}, {"request"=>{"method"=>"GET", "url"=>"/api/mytest"}, "response"=>{"status"=>200, "body"=>"More content"}}, {"request"=>{"method"=>"GET", "url"=>"/api/mytest"}, "response"=>{"status"=>200, "body"=>"More content"}}]}

In [18]:
require 'json'

JSON.generate(mappings)

"{\"mappings\":[{\"request\":{\"method\":\"GET\",\"url\":\"/api/mytest\"},\"response\":{\"status\":200,\"body\":\"More content\"}},{\"request\":{\"method\":\"GET\",\"url\":\"/api/mytest\"},\"response\":{\"status\":200,\"body\":\"More content\"}},{\"request\":{\"method\":\"GET\",\"url\":\"/api/mytest\"},\"response\":{\"status\":200,\"body\":\"More content\"}}]}"

In [19]:
f.write(JSON.pretty_generate(mappings))

553

In [20]:
f.close()

In [9]:
require 'json'

filter_results.each_with_index do |doc, index|
  mapping_file = File.new("./wiremock/mappings/request_mapping-#{index}.json", "w")
  request_file = File.new("./wiremock/__files/request_body-#{index}.json", "w")
  
  json = JSON.pretty_generate({
        :request => doc['request'],
        :response => {
            :status => doc['response']['status'],
            :bodyFileName => File.basename(request_file.path),
            :headers => {
                "Content-type" => "application/json"
              }
          }
      })
  
  puts json
  
  mapping_file.write(json)
  mapping_file.close()
  
  request_file.write(JSON.pretty_generate(
      doc['response']['body']
    ))
  
  request_file.close()
end

{
  "request": {
    "method": "GET",
    "url": "/api/mytest2"
  },
  "response": {
    "status": 200,
    "bodyFileName": "request_body-0.json",
    "headers": {
      "Content-type": "application/json"
    }
  }
}
{
  "request": {
    "method": "GET",
    "url": "/api/mytest"
  },
  "response": {
    "status": 200,
    "bodyFileName": "request_body-1.json",
    "headers": {
      "Content-type": "application/json"
    }
  }
}
{
  "request": {
    "method": "GET",
    "url": "/api/mytest3"
  },
  "response": {
    "status": 200,
    "bodyFileName": "request_body-2.json",
    "headers": {
      "Content-type": "application/json"
    }
  }
}


[{"request"=>{"method"=>"GET", "url"=>"/api/mytest2"}, "response"=>{"status"=>200, "body"=>"More content"}}, {"request"=>{"method"=>"GET", "url"=>"/api/mytest"}, "response"=>{"status"=>200, "body"=>"More content"}}, {"request"=>{"method"=>"GET", "url"=>"/api/mytest3"}, "response"=>{"status"=>200, "body"=>"More content"}}]